In [16]:
import ipywidgets as widgets
from PIL import Image
import io

error_images_ejemplo_1 = [
    '../Basico/imagenes/error/error_image_1.png',
    '../Basico/imagenes/error/error_image_2.png',
    '../Basico/imagenes/error/error_image_3.png',
    '../Basico/imagenes/error/error_image_4.png',
    '../Basico/imagenes/error/error_image_5.png'
]

def create_button(description, color, font_size, font_weight, width='auto'):
    button = widgets.Button(
        description=description,
        layout=widgets.Layout(width=width, padding='2px', margin='5px'),
        style={'button_color': color, 'font_weight': font_weight, 'font_size': font_size, 'text_align': 'center'}
    )
    return button

def display_image(image_path, format='png'):
    with open(image_path, "rb") as f:
        img = Image.open(f)
        img_byte_array = io.BytesIO()
        img.save(img_byte_array, format=format)
        img_data = img_byte_array.getvalue()
    return widgets.Image(value=img_data, format=format)

class BaseQuizApp:
    def __init__(self, quiz_data):
        self.quiz_data = quiz_data
        self.current_question_index = 0
        self.error_count = 0
        self.output_box = widgets.Output()
        self.create_quiz()
        self.display()

    def create_quiz(self):
        pass

    def display(self):
        with self.output_box:
            self.output_box.clear_output()
            display(self.question_label)
            display(self.image_box)  
            display(self.options_box)

class QuizApp_ejemplo_1(BaseQuizApp):
    def __init__(self, quiz_data):
        super().__init__(quiz_data)

    def create_quiz(self):
        self.output_box.clear_output()
        self.option_buttons = []

        if self.current_question_index < len(self.quiz_data):
            quiz = self.quiz_data[self.current_question_index]

            self.question_label = widgets.HTML(
                value=f"<h2 style='color: #0056b3; text-align: center;'>{quiz['question']}</h2>"
            )

            self.option_buttons = [
                create_button(option, '#e8ebe9', '14px', 'bold') for option in quiz['options']
            ]

            for button in self.option_buttons:
                button.on_click(lambda b, opt=button.description: self.on_button_clicked(opt, quiz['correct_option']))

            with open('../Basico/imagenes/base/base.png', 'rb') as f:
                self.image_widget = widgets.Image(
                    value=f.read(),
                    format='png', 
                    layout=widgets.Layout(max_width='700px', max_height='700px', object_fit='contain')
                )
            self.image_box = widgets.HBox([self.image_widget], layout=widgets.Layout(justify_content='center'))

            self.options_box = widgets.VBox(self.option_buttons, layout=widgets.Layout(align_items='center'))

        else:
            self.question_label = widgets.HTML(
                value="<h2 style='color: #28a745; text-align: center;'>¡Bien hecho!</h2>"
            )
            self.options_box = widgets.VBox([])
            self.image_box = widgets.VBox([])

    def on_button_clicked(self, option, correct_option):
        if option == correct_option:
            self.show_message("Correcto", "¡Buena!")
            self.next_question()
        else:
            self.error_count += 1
            message = self.get_error_message()
            self.show_error_message(message)

    def get_error_message(self):
        if self.error_count < 2:
            return "Whops, esa no era la respuesta correcta. Prueba otra vez."
        elif self.error_count < 4:
            return "¿Estás seguro de que has atendido a la explicación?"
        elif self.error_count < 6:
            return "Estás haciendo al búho llorar."
        elif self.error_count < 7:
            return "¿Te diviertes?"
        else:
            return "..."

    def show_message(self, title, message):
        with self.output_box:
            self.output_box.clear_output()
            display(widgets.HTML(f"<h3 style='text-align: center;'>{title}: {message}</h3>"))


    def show_error_message(self, message):
        with self.output_box:
            self.output_box.clear_output()
    
            error_img_path = self.get_error_image_path()
    
            with open(error_img_path, "rb") as file:
                error_img = widgets.Image(
                    value=file.read(),
                    format='png',  
                    layout=widgets.Layout(max_width='500px', max_height='400px', object_fit='contain')
                )
    
            error_button = create_button("Venga voy", '#f44336', '14px', 'bold')
            error_button.on_click(self.retry_current_question)
    
            display(widgets.VBox([
                widgets.HBox([error_img], layout=widgets.Layout(justify_content='center')),
                widgets.HTML(f"<h4 style='text-align: center;'>{message}</h4>"),
                error_button
            ], layout=widgets.Layout(align_items='center')))

    def retry_current_question(self, b):
        self.create_quiz()
        self.display()

    def get_error_image_path(self):
        index = min(self.error_count - 1, len(error_images_ejemplo_1) - 1)
        return error_images_ejemplo_1[index]

    def next_question(self):
        self.current_question_index += 1
        self.create_quiz()
        self.display()

def run_quiz(quiz_data):
    quiz_app = QuizApp_ejemplo_1(quiz_data)
    display(quiz_app.output_box)


<!DOCTYPE html>
<html lang="es">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
<style>
    body {
        font-family: Arial, sans-serif;
        background-color: #000000;
        margin: 0;
        padding: 0;
    }
    .content {
        max-width: 90%;
        width: 150ch;
        margin: 0 auto;
        padding: 20px;
        background-color: rgba(255, 255, 255, 0.8); 
        box-shadow: 0 0 20px rgba(0, 0, 0, 0.5);
        border-top: 1px solid rgba(0, 0, 0, 0.5);
    }
    h1, h2, h3 {
        color: #333;
    }
    a {
        color: #1a73e8;
        text-decoration: none;
    }
    a:hover {
        text-decoration: underline;
    }
    em {
        font-style: italic;
    }
    strong {
        font-weight: bold;
    }
    @media (max-width: 1290px) {
        .content {
            max-width: 80%; 
            padding: 20px;
            width: 120ch; 
        }
    }
    @media (max-width: 480px) {
        .content {
            max-width: 100%;
            padding: 20px; 
        }
    }
</style>
</head>
<body>
    <div class="content">
       <h1 id="introducci-n-a-los-sistemas-basados-en-reglas">Introducción a los Sistemas Basados en Reglas</h1>
<p>Los sistemas basados en reglas son sistemas basados en reglas y la aplicación de las mismas. <br>
Eso es todo, gracias por venir a mi charla TED.<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
Ahora en serio, los sistemas basados en reglas son un tipo de sistema, frecuentemente utilizados en contextos de Inteligencia Artificial, en dónde el conocimiento relativo al dominio se representa mediante reglas a partir de las cuales se extraen conclusiones sobre el problema a resolver.<br>
Los componentes comunes a todos los SBR son la <strong>base de conocimiento</strong> y el <strong>motor de inferencia</strong> (del que hablaremos más adelante).<br>
La base de conocimiento contendrá todo el conocimiento, valga la redundancia, relativo al dominio en el que nos encontramos mediante <strong>reglas</strong> y <strong>hechos</strong>.<br>
¿Recuerdas el “modus ponendo ponens” del primer capítulo? Más te vale, porque está <strong>tres</strong> páginas atrás, desgraciado.<br>
Pues eso son las reglas y hechos en Prolog:
Hechos: Afirmaciones que se asumen verdaderas.
Reglas: Premisas &quot;si... entonces&quot;, con antecedente y consecuente.<br>
Prolog no es más que un gigantesco sistema de inferencia donde “deduce” (infiere) la validez de una conclusión a partir de reglas, hechos y un objetivo.<br></p>
<h2 id="hechos">Hechos</h2>
<p>Los hechos representan cuestiones conocidas sobre el dominio y que sabemos verdaderas, como que el agua moja o que el Sol quema. En prolog los hechos se escriben en forma de cláusulas de la siguiente manera:<br>
moja(agua).<br>
quema(sol).<br>
Tenemos por una parte las cláusulas &quot;moja&quot; y &quot;quema&quot;, y sus respectivos argumentos. Si nosotros lanzásemos una consulta del tipo &quot;moja(arena).&quot;, el SBR determinaría que es falso, al no encontrar una coincidencia en la base de conocimiento.<br>
Un dato importante, ¡nuestros hechos pueden contener tantos argumentos como necesitemos para representar lo que buscamos! Por ejemplo, si quisiéramos definir un árbol genealógico con hechos, podríamos representarlo de la siguiente manera:<br>
        padre(anton, mario).<br>
        madre(sofia, mario).<br>
        padre(mario, juan).<br>
        abuelo(anton, juan).<br>
        abuela(sofia, juan).<br>
El problema de representar el conocimiento de esta manera es que puede llegar a ser extremadamente costoso e ineficiente si el dominio es muy extenso, dado que estamos tratando a la base de conocimiento como una base de datos venida a menos. Aquí es donde entran las reglas.<br></p>
<h2 id="reglas">Reglas</h2>
<p>Las reglas representan relaciones entre elementos abstractos, son un mecanismo de deducción. En lugar de utilizar elementos concretos del dominio, trabajamos con abstracciones o variables generales para definir condiciones que deben cumplirse para que así la regla sea verdadera, lo cual nos permite obtener nuevo conocimiento a partir de lo que ya sabemos o representar el conocimiento existente de manera más compacta.<br>
En Prolog, las reglas se escriben en siguiendo las pautas de los predicados de primer orden. Son condicionales en donde la parte de la izquierda será verdadera si se cumple lo estipulado en la parte derecha. <br>
Volviendo al ejemplo anterior, podemos reemplazar el hecho &quot;abuelo&quot; con una regla &quot;abuelo&quot; de la siguiente manera:<br>
abuelo(Persona1, Persona2):-
        padre(Persona1, X),
        padre(X, Persona2).<br>
En esta relación, Persona1 será abuelo de Persona2 si Persona1 es padre de X y X es padre de Persona2.<br>
El uso de reglas nos permite reducir enormemente la cantidad de hechos a representar, ya que en lugar de especificar todo el dominio, podemos moderlarlo mediante estas relaciones, disminuyendo enormemente la cantidad de hechos que aparecerán en la base de conocimiento.<br></p>
<p>Podemos encadenar tantas reglas como queramos para definir las relaciones que necesitemos modelar, pudiendo referenciar a otros predicados o ejecutarse de manera recursiva.<br></p>
<h2 id="objetivo">Objetivo</h2>
<p>Las reglas y hechos forman la base de conocimiento de nuestro programa Prolog, pero por desgracia por si solos sirven para lo mismo que un pañal en medio del mar. Pa ná. Como aprenderse memoria los 32 volúmenes de la última edición de la enciclopedia británica sin saber inglés. Prolog es, fundamentalmente, mucho conocimiento estático que no hace nada.<br></p>
<p>¿Cómo hacemos que Prolog haga algo? Tratándolo como a un semejante y preguntándole cosas.(Puntos extra si dices <strong>por favor</strong>).<br></p>
<p>El objetivo es sencillamente esto, una pregunta. En el momento en el que se formula la misma (objetivo), Prolog, haciendo uso del motor de inferencia, busca en su descomunal base de conocimiento algo que la conteste, algo que demuestre que es <strong>verdadera</strong>.<br></p>
</body>
</html>


In [15]:


quizz_1 = [
    {
        'question': '¿Qué componente de un Sistema Basado en Reglas (SBR) contiene todo el conocimiento relativo al dominio?',
        'options': [
            'El motor de inferencia.',
            'La base de conocimiento.',
            'Las reglas.',
            'Los hechos.'
        ],
        'correct_option': 'La base de conocimiento.',
    },
    {
        'question': '¿Qué representan los hechos en Prolog?',
        'options': [
            'Cuestiones sobre el dominio del problema que sabemos verdaderas.',
            'Afirmaciones que se asumen verdaderas en forma de cláusulas.',
            'Variables abstractas en el motor de inferencia.',
            'Predicados de primer orden con operadores lógicos.'
        ],
        'correct_option': 'Cuestiones sobre el dominio del problema que sabemos verdaderas.',
    },
    {
        'question': '¿Qué representan las reglas en Prolog?',
        'options': [
            'Cuestiones mucho más detalladas que los hechos sobre el dominio del problema que sabemos verdaderas.',
            'Son condicionales que definen relaciones entre elementos abstractos y permiten inferir conocimiento sin desarrollarlo explícitamente o, incluso, definir nuevo conocimiento.',
            'Son procedimientos que especifican el flujo de ejecución de un programa.',
            'Son condicionales estáticos que se utilizan para almacenar información sobre el dominio.'
        ],
        'correct_option': 'Son condicionales que definen relaciones entre elementos abstractos y permiten inferir conocimiento sin desarrollarlo explícitamente o, incluso, definir nuevo conocimiento.',
    },
    {
        'question': '¿Cuál es una de las ventajas de usar reglas en lugar de representar todo con hechos en Prolog?',
        'options': [
            'Permite representar el conocimiento de manera más extensa, detallada y menos confusa.',
            'Disminuye la cantidad de hechos necesarios al modelar relaciones en lugar de definir todo el conocimiento explícitamente.',
            'Reduce la necesidad de utilizavariables en las consultas.',
            'Aumenta la eficiencia de búsqueda en la base de conocimiento mediante consultas directas.'
        ],
        'correct_option': 'Disminuye la cantidad de hechos necesarios al modelar relaciones en lugar de definir todo el conocimiento explícitamente.',
    },
    
    {
        'question': '¿Qué es el objetivo en un programa Prolog?',
        'options': [
            'La afirmación que se intenta probar verdadera en la base de conocimiento.',
            'La regla que define cómo se deben deducir nuevo conocimiento.',
            'La pregunta a la que el motor de inferencia busca respuesta en la base de conocimiento.',
            'Un dato estático que determina si el resultado es correcto.'
        ],
        'correct_option': 'La pregunta a la que el motor de inferencia busca respuesta en la base de conocimiento.',
    }
]
run_quiz(quizz_1)


Output()

<!DOCTYPE html>
<html lang="es">
<body>
    <div class="content">
<h3 id="-siguiente-cap-tulo-backtracking-composicion-backtracking-ipynb-">- <a href="../Composicion/Backtracking.ipynb">Siguiente capítulo - Backtracking</a></h3>
<h3 id="-apartado-anterior-paradigmas-de-programaci-n-paradigmas-ipynb-">- <a href="Paradigmas.ipynb">Apartado anterior - Paradigmas de programación</a></h3>
<h3 id="-volver-al-ndice-indice-ipynb-">- <a href="../Indice.ipynb">Volver al índice</a></h3>

</body>
</html>